In [2]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [3]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [ ]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

In [ ]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

In [ ]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [14]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [1]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

NameError: name 'reload' is not defined

In [ ]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

In [ ]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

In [ ]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [17]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.203s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [23]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL, little_endian_to_int, hash256
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = [
    'myRo6mdjL1o1KuS8tiA4cnkmQCsTReqEfz', #내 테스트 지갑 주소
    'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
]

# get the private key from the exercise in Chapter 4
passphrase = b'netscout82@naver.com wannabes software factory and onlifecoding youtube channel'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)

# change address should be the address generated from Chapter 4

# get the prev_tx and prev_index from the transaction where you got
#75a1c4bc671f55f626dda1074c7725991e6f68b8fcefcfca7b64405ca3b45f1c
prev_tx = bytes.fromhex('88346d3cb62a95693092b3cde472493ee64a04e7a9444b602773d4512890f54a')
prev_index = 1

# some testnet coins
# create a transaction input for the previous transaction with
# the default ScriptSig and sequence
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []

# target amount should be 60% of the output amount
# set the fee to some reasonable amount
# change amount = amount from the prev tx - target amount - fee
total_amount = tx_in.value(True)

# create a transaction output for the target amount and address
target_amount = int(total_amount * 0.6)
target_h160 = decode_base58(target_address[1]) #보낼 주소
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)

# create a transaction output for the change amount and address
change_amount = int(total_amount * 0.3)
change_h160 = decode_base58(target_address[0]) #내 주소
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)

# create the transaction object
#version, tx_ins, tx_outs, locktime, testnet
tx = Tx(1, [tx_in], [change_output, target_output], 0, True)

# sign the one input in the transaction object using the private key
tx.sign_input(0, private_key)

# print the transaction's serialization in hex
print(tx.serialize().hex())

#아래 주소로 변경되었음.
# broadcast at https://live.blockcypher.com/btc-testnet/pushtx/

01000000014af5902851d47327604b44a9e7044ae63e4972e4cdb3923069952ab63c6d3488010000006b483045022100c2eca01774233d83eaba30810f31c0ad7e84dbff251e96508b3d3e7e3353b72a0220565bc3766cea1e54f21b9dbbde66d9f545c930d4a0160ddd7e91553d2dc176450121032ac827b9d0a5e8a5cc47aa34587bd3b2d68448913ed89654aaed9884779d7216ffffffff02e0930400000000001976a914c4788252f9d4c57b3d05243bd5ec70663a19b4c188acc0270900000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac00000000


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [27]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
input_addresses = [
    'mfc8kiqKbkxiQPaKLBgT1WzA6b5AsUesV6'
    'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
]
# Creat 1 TxOut to the address above
target_address = 'mfc8kiqKbkxiQPaKLBgT1WzA6b5AsUesV6'

# get the private key from the exercise in Chapter 4

#새로 생성한 테스트 주소
passphrase = b'kangboram2@gmail.com wannabes software factory and onlifecoding youtube channel'
# secret = little_endian_to_int(hash256(passphrase))
secret = little_endian_to_int(hash256(passphrase))
# create a private key using your secret
private_key1 = PrivateKey(secret)


#2번째 주소는 이전에 생성한 테스트 주소
passphrase = b'netscout82@naver.com wannabes software factory and onlifecoding youtube channel'
secret = little_endian_to_int(hash256(passphrase))
private_key2 = PrivateKey(secret)


# get the prev_tx and prev_index from the transaction where you got
tx_in1 = TxIn(
    bytes.fromhex('7de46a40d89c9f1092e87c9b8b8a1cea0444d1fae6390e27a0036bf2ef92601c'), 
    1)

tx_in2 = TxIn(
    bytes.fromhex('9d966e955f9d9f7a4ba6a58d23ea7fcc17dd7395b02dd71c2bb3507eaeb06621'), 
    0)

# some testnet coins
# create a transaction input for the previous transaction with
# the default ScriptSig and sequence
tx_ins = [
    tx_in1, tx_in2
]
tx_outs = []

# target amount should be 60% of the output amount
# set the fee to some reasonable amount
# change amount = amount from the prev tx - target amount - fee
total_amount = tx_in1.value(True) + tx_in2.value(True)

# create a transaction output for the change amount and address
target_amount = int(total_amount * 0.9)
target_h160 = decode_base58(target_address) #새로 생성한 테스트 주소
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)

# create the transaction object
#version, tx_ins, tx_outs, locktime, testnet
tx = Tx(1, tx_ins, [target_output], 0, True)

# sign the one input in the transaction object using the private key
tx.sign_input(0, private_key1)
tx.sign_input(1, private_key2)

# print the transaction's serialization in hex
print(tx.serialize().hex())

#아래 주소로 변경되었음.
# broadcast at https://live.blockcypher.com/btc-testnet/pushtx/

#전송 결과
#두 번째 테스트 주소 : https://live.blockcypher.com/btc-testnet/address/mfc8kiqKbkxiQPaKLBgT1WzA6b5AsUesV6/
#첫 번째 테스트 주소 : https://live.blockcypher.com/btc-testnet/address/myRo6mdjL1o1KuS8tiA4cnkmQCsTReqEfz/

01000000021c6092eff26b03a0270e39e6fad14404ea1c8a8b9b7ce892109f9cd8406ae47d010000006a473044022013e4cdda0c7e5f6708b1b44ba14a5d3dd4774a90808517137e219eec88cd9c8e022066bd3836805a05f865faa9c4c334ae39554045816784eecd5bffbc803523cfd90121027f3aa33ea15ded16f643d9cae03aa3cca09a14dc6537a1cc2a8e060282802fc2ffffffff2166b0ae7e50b32b1cd72db09573dd17cc7fea238da5a64b7a9f9d5f956e969d000000006b48304502210089b290ff7e50610d71d1f7b99eb10cd7a59174a9b182e5f2237333054ad4130102203c1a8ae6d95f3709cdb0ceb026e30b95e5f398f23ce0253eb2d61d9389bdd0ec0121032ac827b9d0a5e8a5cc47aa34587bd3b2d68448913ed89654aaed9884779d7216ffffffff0150da1100000000001976a91400facfab2704370b02b829642538e5ea7b20c11e88ac00000000
